<a href="https://colab.research.google.com/github/strongbyson/Quant-Projects/blob/main/Final_vers_UVXY_VIX_trading_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: import all requisite libraries to create a trading strategy

!pip install yfinance pandas numpy matplotlib

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# prompt: load and read 2 csv files as dataframes

# Download the CSV files (replace with your file paths)
trade_data = pd.read_csv('Trade Data.csv')

# Print some info
print(trade_data.head(10))


      Date    vix       uvxy 
0  02/01/19  27.54  43,000.00
1  02/01/19  28.53  43,240.00
2  02/01/19  23.05  38,715.00
3  02/01/19  23.22  38,960.00
4  03/01/19  25.68  40,155.00
5  03/01/19  26.60  42,977.50
6  03/01/19  24.05  39,950.00
7  03/01/19  25.45  41,650.00
8  04/01/19  24.36  39,125.00
9  04/01/19  24.48  39,605.00


In [102]:
# prompt: Now we want to build a trading strategy as per rules below.
# 1. initialize with total trading capital of $1,000,000. First trade amount will be $10000.
# Available for trade capital is always 80% of total trading capital.
# Units traded starts at 0
# 2. Create a Trade counter, which will start at 0, and keep a count of number of traders executed. whenever a short trade is executed it adds to the existing counter and whenever a buy back order is executed, the trade counter resets.
# 2.create 3 conditions of vix, where vix  is above 20, below 19, and between 19 and 20.
# 3. if vix is above 20, check if trade counter is more than 0, if its not, trade amount will be standard trade amount of $10000, if trade counter is more than 0, trade amount will be = Trade counter * standard trade amount.
# 4. check if the trade amount is less than the available for trade capital, if no, loop it back to initial step of VIX scanning, if it is less than available for trade capital, check UVXY price on that row and calculate the  "units traded" which is Trade amount/uvxy price. add units traded to "number of units traded counter". and add trade amount to Initial capital. Now loop it back to initial step of VIX scanning.
# 5. if VIX is less than 19, check total number of units traded and check current uvxy price and calculate buy units by multiplying units traded to uvxy price. the resultant amount will be deducted from initial capital amount. units bought will be deducted from total number of units traded. and then loop it back to initial step of vix scanning.
# 6. if vix is between 19 and 20, just keep, loop it back to initial step of vix scanning.

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## prompt: load and read 2 csv files as dataframes

# Download the CSV files (replace with your file paths)
trade_data = pd.read_csv('Trade Data.csv')

# Print some info
#print(trade_data.head(10))

# Initialize trading parameters
total_demat_capital = 1000000
available_capital_to_trade = total_demat_capital * 0.8
standard_trade_amount = 10000
trade_counter = 0
units_traded_counter = 0
prev_vix = 20
cumulative_units_traded = 0
cumulative_sell_amount = 0

# Remove whitespaces from all columns
trade_data.columns = trade_data.columns.str.strip()
# Remove commas from the 'uvxy_price' column and convert to float
trade_data['uvxy'] = trade_data['uvxy'].astype(str).str.replace(',', '').astype(float)

# Add new columns to the dataframe
trade_data['prev_vix'] = np.nan
trade_data['trade_counter'] = np.nan
trade_data['sell_amount'] = np.nan
trade_data['cumulative_sell_amount'] = np.nan
trade_data['available_capital_to_trade'] = np.nan
trade_data['units_traded'] = np.nan
trade_data['cumulative_units_traded'] = np.nan
trade_data['buy_back_amount'] = np.nan
trade_data['total_demat_capital'] = np.nan
trade_data['profit'] = np.nan

# Iterate through the trade data
for index, row in trade_data.iterrows():
  vix = row['vix']
  uvxy_price = row['uvxy']

  # VIX above 20: Short UVXY
  if (vix > prev_vix + 0.5) and (vix > 20):
    if trade_counter == 0:
      sell_amount = standard_trade_amount
    else:
      sell_amount = (trade_counter + 1) * standard_trade_amount

    # Checking if sell amount is under available capital to trade
    if sell_amount <= available_capital_to_trade:
      # Calculating units to be traded
      units_traded = sell_amount / uvxy_price
      # Calcuting cumulative units traded
      cumulative_units_traded += units_traded
      # Keeping track of number of trades executed
      trade_counter += 1
      # Updating new VIX value
      prev_vix = vix
      # Calculating cumulative sell amount
      cumulative_sell_amount += sell_amount
      # Calculating available capital to trade
      available_capital_to_trade = available_capital_to_trade - sell_amount
      # Updating total demat capital
      total_demat_capital = total_demat_capital + sell_amount


      trade_data.at[index, "prev_vix"] = prev_vix
      trade_data.at[index, "trade_counter"] = trade_counter
      trade_data.at[index, 'sell_amount'] = sell_amount
      trade_data.at[index, "cumulative_sell_amount"] = cumulative_sell_amount
      trade_data.at[index, "available_capital_to_trade"] = available_capital_to_trade
      trade_data.at[index, "units_traded"] = units_traded
      trade_data.at[index, "cumulative_units_traded"] = cumulative_units_traded
      #trade_data.at[index, "total_demat_capital"] = total_demat_capital


  # VIX below 19: Buy back UVXY
  elif vix < 19:
    # Check if there are existing trades going on
    if cumulative_units_traded > 0:
      # Update sell amount column to nullify it
      sell_amount = -(cumulative_sell_amount)
      # Update available capital to trade
      available_capital_to_trade = available_capital_to_trade - sell_amount
      # Calculate the buyback amount
      buy_back_amount = cumulative_units_traded * uvxy_price
      # Updating total demat capital
      total_demat_capital = total_demat_capital - buy_back_amount
      prev_vix = vix
      cumulative_units_traded = 0
      trade_counter = 0
      cumulative_sell_amount = 0
      # Calculate profit
      profit = total_demat_capital - 1000000



      trade_data.at[index, "prev_vix"] = prev_vix
      trade_data.at[index, "trade_counter"] = trade_counter
      trade_data.at[index, 'sell_amount'] = sell_amount
      trade_data.at[index, 'buy_back_amount'] = buy_back_amount
      trade_data.at[index, "total_demat_capital"] = total_demat_capital
      trade_data.at[index, "available_capital_to_trade"] = available_capital_to_trade
      trade_data.at[index, "profit"] = profit


  # VIX between 19 and 20: Do nothing
  else:
    pass




    trade_data.at[index, 'cumulative_units_traded'] = cumulative_units_traded


print(trade_data.head(40))
trade_data.to_csv('trade_data_export.csv', index=False)


        Date    vix      uvxy  prev_vix  trade_counter  sell_amount  \
0   02/01/19  27.54  43000.00     27.54            1.0      10000.0   
1   02/01/19  28.53  43240.00     28.53            2.0      20000.0   
2   02/01/19  23.05  38715.00       NaN            NaN          NaN   
3   02/01/19  23.22  38960.00       NaN            NaN          NaN   
4   03/01/19  25.68  40155.00       NaN            NaN          NaN   
5   03/01/19  26.60  42977.50       NaN            NaN          NaN   
6   03/01/19  24.05  39950.00       NaN            NaN          NaN   
7   03/01/19  25.45  41650.00       NaN            NaN          NaN   
8   04/01/19  24.36  39125.00       NaN            NaN          NaN   
9   04/01/19  24.48  39605.00       NaN            NaN          NaN   
10  04/01/19  21.19  36591.50       NaN            NaN          NaN   
11  04/01/19  21.38  36770.00       NaN            NaN          NaN   
12  07/01/19  22.06  36200.00       NaN            NaN          NaN   
13  07